In [1]:
# Visualize the vector representations of the images in the dataset

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np

# Load the vector representations
df = pd.read_csv('/Users/rz20505/Documents/training_year/applied_data_science/models/yolov5-master/data/vectors.csv')


In [2]:
X = df['feature_vector'].values


In [3]:
# The elements of X are string representations of lists of floats
# Convert them to np arrays of floats

for i in range(len(X)):
    X[i] = X[i].strip('][').split(', ')
    X[i] = [float(x) for x in X[i]]

In [4]:
type(X[0])

list

In [5]:
# convert each of the lists in X to a numpy array
X = np.array([np.array(xi) for xi in X])

In [6]:
import numpy as np

def create_sprite(data):
    """
    Tile images into sprite image. 
    Add any necessary padding
    """
    
    # For B&W or greyscale images
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0), (0, 0), (0, 0))
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    
    # Tile images into sprite
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3, 4))
    # print(data.shape) => (n, image_height, n, image_width, 3)
    
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    # print(data.shape) => (n * image_height, n * image_width, 3) 
    return data

In [7]:
import os
import glob

import cv2

LOG_DIR = "/Users/rz20505/Documents/training_year/applied_data_science/models/yolov5-master/my_testing/tensorboard-logs/"
IMAGES_DIR = "/Users/rz20505/Documents/training_year/applied_data_science/data/uob_image_set_resized/"
IMAGE_SIZE = (64, 64)
SPRITES_FILE = os.path.join(LOG_DIR, "sprites.png")

# Max sprite size is 8192 x 8192 so this max samples makes visualization easy
MAX_NUMBER_SAMPLES = 8191 
image_files = glob.glob(os.path.join(IMAGES_DIR, "*/*.jpg"))

img_data = []
for img in image_files[:MAX_NUMBER_SAMPLES]:
    input_img = cv2.imread(img)
    input_img_resize = cv2.resize(input_img, IMAGE_SIZE) 
    img_data.append(input_img_resize)
img_data = np.array(img_data)

sprite = create_sprite(img_data)
cv2.imwrite(SPRITES_FILE, sprite)

True

In [8]:
# load the feature vectors, generate metadata, and save them for loading into TensorBoard:

import tensorflow as tf

# define projector
from tensorboard.plugins import projector

# Create a metadata file
with open(os.path.join(LOG_DIR, "metadata.tsv"), "w") as f:
    # f.write("Index\tLabel\n")
    for index, label in enumerate(df['image'].loc[:MAX_NUMBER_SAMPLES]):
        # strip the path of the image /Users/rz20505/Documents/training_year/applied_data_science/data/uob_image_set_resized/11059585/
        label = label.split("/")[-1]
        # remove the .jpg
        label = label.split(".")[0]
        label = label.split("_")[0]
        f.write("{}\n".format(label))

# prepare the embeddings for TensorBoard
embedding_var = tf.Variable(X[:MAX_NUMBER_SAMPLES], name="embedding")

# save the embedding variable
checkpoint = tf.train.Checkpoint(embedding=embedding_var)
checkpoint.save(os.path.join(LOG_DIR, "embedding.ckpt"))

# Create a config file
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = "metadata.tsv"
embedding.sprite.image_path = "sprites.png"
embedding.sprite.single_image_dim.extend(IMAGE_SIZE)
projector.visualize_embeddings(LOG_DIR, config)

2023-03-31 14:39:07.091086: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 14:39:23.653832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
embedding_var

<tf.Variable 'embedding:0' shape=(6411, 1500) dtype=float64, numpy=
array([[  2.89594746,  -9.57545662,  -7.81872129, ...,  -7.29773617,
         -6.42428207,  -4.91425228],
       [ -0.41966534, -11.16795731, -10.74712181, ...,  -8.68289948,
         -9.22375965,  -8.1003046 ],
       [  1.41287422,  -9.68728542,  -8.31381798, ...,  -7.12037849,
         -6.57165861,  -5.75563526],
       ...,
       [ -3.61702681,  -6.38735104,  -6.57878208, ...,  -6.9664216 ,
         -6.21995401,  -0.63406545],
       [ -2.69703794,  -5.46993256,  -5.3597002 , ...,  -5.80486584,
         -5.3774209 ,   1.04141784],
       [ -8.7504406 ,  -5.02199125, -10.52400208, ...,  -8.0613575 ,
         -7.71609545,   0.90739143]])>

In [12]:
# Run tensorboard
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

Reusing TensorBoard on port 6006 (pid 58091), started 0:00:08 ago. (Use '!kill 58091' to kill it.)